<a href="https://colab.research.google.com/github/Farmhouse121/Wilmott-Articles/blob/main/Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
print("Installing yfinance and arch and getting the data...")
!pip install arch 1>/dev/null
!pip install yfinance 1>/dev/null
from warnings import filterwarnings ; filterwarnings('ignore',category=FutureWarning) # I don't care
from yfinance import download
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from statsmodels.regression.linear_model import OLS
from statsmodels.api import add_constant
from arch.univariate import ARX, GARCH, GeneralizedError
from datetime import datetime
zero,one,two,four,three,five,ten,hundred=0e0,1e0,2e0,3e0,4e0,5e0,1e1,1e2 # some friendly numbers
half,GoldenRatio=one/two,(one+np.sqrt(five))/two

# get the daily returns from Yahoo! Finance
tickers=['DIA','SPY','QQQ']
data=download(tickers,'1999-03-11','2009-12-31') # this date covers all tickers
data.index=pd.DatetimeIndex(data.index).to_period('B')  # set calendar to business days

for ticker in tickers:
    data[('Return',ticker)]=data[('Adj Close',ticker)].pct_change()*hundred   # compute returns including dividends and splits etc.
    data[('LagReturn',ticker)]=data[('Return',ticker)].shift()                # compute returns lagged by one trading day

data.dropna(inplace=True)                               # remove any dates that are not properly 'bars', often due to holidays not properly removed
data

Installing yfinance and arch and getting the data...
[*********************100%***********************]  3 of 3 completed


Adj Close                             Close             \
                  DIA        QQQ        SPY         DIA        QQQ   
Date                                                                 
1999-03-15  59.734192  44.388474  85.466309   99.734375  51.500000   
1999-03-16  59.556381  44.765545  85.140648   99.437500  51.937500   
1999-03-17  59.247517  44.442322  84.774292   98.921875  51.562500   
1999-03-18  60.080460  45.304230  86.138008  100.312500  52.562500   
1999-03-19  59.120754  44.146027  84.673836   98.562500  51.218750   
...               ...        ...        ...         ...        ...   
2009-12-23  78.461349  40.404583  87.778336  104.449997  45.560001   
2009-12-24  78.874535  40.777050  88.193916  105.000000  45.980000   
2009-12-28  79.092354  40.989906  88.382095  105.290001  46.220001   
2009-12-29  79.092354  40.821411  88.256645  105.290001  46.029999   
2009-12-30  79.039772  40.945553  88.225266  105.220001  46.169998   

                              High                                Low  ...  \
                   SPY         DIA        QQQ         SPY         DIA  ...   
Date                                                                   ...   
1999-03-15  131.218750   99.750000  51.562500  131.250000   98.750000  ...   
1999-03-16  130.718750  100.109375  52.156250  131.656250   99.265625  ...   
1999-03-17  130.156250   99.750000  52.000000  130.937500   98.687500  ...   
1999-03-18  132.250000  100.468750  52.593750  132.375000   98.625000  ...   
1999-03-19  129.687500  100.750000  53.250000  132.625000   98.562500  ...   
...                ...         ...        ...         ...         ...  ...   
2009-12-23  111.949997  104.720001  45.570000  112.110001  104.169998  ...   
2009-12-24  112.480003  105.040001  45.980000  112.599998  104.550003  ...   
2009-12-28  112.720001  105.349998  46.299999  112.989998  104.889999  ...   
2009-12-29  112.559998  105.639999  46.250000  113.029999  105.250000  ...   
2009-12-30  112.519997  105.330002  46.259998  112.650002  104.879997  ...   

                  Open   Volume                         Return LagReturn  \
                   SPY      DIA       QQQ        SPY       DIA       DIA   
Date                                                                       
1999-03-15  129.937500   204000   6369000    5394400  1.108784 -0.676540   
1999-03-16  131.125000   644400   4905800    4547500 -0.297670  1.108784   
1999-03-17  130.687500   608100   3965000    4524100 -0.518609 -0.297670   
1999-03-18  129.781250   399800   4848400    3506300  1.405870 -0.518609   
1999-03-19  132.312500   501000   7160400    5526700 -1.597367  1.405870   
...                ...      ...       ...        ...       ...       ...   
2009-12-23  112.000000  4867200  73932200  111783100  0.000000  0.548721   
2009-12-24  112.190002  3568000  31770400   39677500  0.526610  0.000000   
2009-12-28  112.900002  8116800  49381400   87508500  0.276159  0.526610   
2009-12-29  113.010002  3356400  32410200   80572500  0.000000  0.276159   
2009-12-30  112.230003  3243500  57612700   73138400 -0.066482  0.000000   

              Return LagReturn    Return LagReturn  
                 SPY       SPY       QQQ       QQQ  
Date                                                
1999-03-15  1.425118 -0.956952  2.871428 -2.436068  
1999-03-16 -0.381040  1.425118  0.849480  2.871428  
1999-03-17 -0.430295 -0.381040 -0.722035  0.849480  
1999-03-18  1.608643 -0.430295  1.939386 -0.722035  
1999-03-19 -1.699798  1.608643 -2.556501  1.939386  
...              ...       ...       ...       ...  
2009-12-23  0.196923  0.359252  0.729579  0.600555  
2009-12-24  0.473443  0.196923  0.921844  0.729579  
2009-12-28  0.213370  0.473443  0.522000  0.921844  
2009-12-29 -0.141941  0.213370 -0.411065  0.522000  
2009-12-30 -0.035555 -0.141941  0.304109 -0.411065  

[2718 rows x 24 columns]

In [9]:
data["Return"].corr()

,DIA,SPY,QQQ
DIA,1.000000,0.952154,0.714138
SPY,0.952154,1.000000,0.808148
QQQ,0.714138,0.808148,1.000000
